In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

## GPT-4 128k Context Length
* Changes RAG: Now you can load one book at a time.

In [ ]:
import openai

with open("my_long_document.txt") as f:
    book_text = f.read()

response = openai.chat.completions.create(
    model="gpt-4-1106-preview",
    messages=[
        {
            "role": "user",
            "content": book_text + "My question about the document?"
        }
    ]
)

print(response.choices[0].message.content)

print(f"\nTotal tokens: {response.usage.total_tokens}")

## Multiple Function Calling

In [ ]:
import openai

messages = [
    {
        "role": "user",
        "content": input("Message: ")
    }
]

tools = [
    {
        "type": "function",
        "function": {
            "name": "play_song",
            "description": "Play a song",
            "parameters": {
                "type": "object",
                "properties": {
                    "song_name": {
                        "type": "string",
                        "description": "The name of the song to play",
                    },
                },
                "required": ["song_name"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "light_dimmer",
            "description": "Adjust the light dimmer from 0-100",
            "parameters": {
                "type": "object",
                "properties": {
                    "brightness": {
                        "type": "number",
                        "description": "The brightness from 0-100",
                    },
                },
                "required": ["brightness"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "order_food",
            "description": "Order food from a restaurant",
            "parameters": {
                "type": "object",
                "properties": {
                    "dish_name": {
                        "type": "string",
                        "description": "The name of the dish to order",
                    },
                    "count": {
                        "type": "number",
                        "description": "The number of dishes to order",
                    },
                },
                "required": ["dish_name", "count"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "send_sms",
            "description": "Send a text message to a contact",
            "parameters": {
                "type": "object",
                "properties": {
                    "contact_name": {
                        "type": "string",
                        "description": "The name of the contact",
                    },
                    "message": {
                        "type": "string",
                        "description": "The text message content",
                    },
                },
                "required": ["contact_name", "message"],
            },
        },
    }
]

response = openai.chat.completions.create(
    #model="gpt-3.5-turbo-1106",
    model="gpt-4-1106-preview",
    messages=messages,
    tools=tools,
    tool_choice="auto",
)

response_message = response.choices[0].message

tool_calls = response_message.tool_calls

messages.append(response_message)

for tool_call in tool_calls:
    function_name = tool_call.function.name
    arguments = tool_call.function.arguments
    print(f"Called '{function_name}' with args {arguments}")


## JSON Mode
* Changes: the most popular Output Parser is not needed anymore.

In [ ]:
import openai
import json

response = openai.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=[
        {
            "role": "system",
            "content": "You are a book suggester. Respond with a book title, description and category in JSON format."
        },
        {
            "role": "user",
            "content": "I want to make lots of money"
        }
    ],
    response_format={"type": "json_object"}
)

print(
    json.dumps(
        json.loads(
            response.choices[0].message.content
        ),
        indent=4
    )
)

## Reproducible Outputs (Seed Parameter)

In [ ]:
import openai

response = openai.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    seed=3453,
    messages=[
        {
            "role": "user",
            "content": "Create a story about a boy and his dog. Give them names"
        }
    ],
    max_tokens=256,
)

print(response.choices[0].message.content)


## Assistants

In [ ]:
import openai

assistant = openai.beta.assistants.create(
    model="gpt-3.5-turbo-1106",
    name="Riddlebot",
    instructions="You answer all questions in riddles",
)

print(f"Assistant ID: {assistant.id}")

thread = openai.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "How do I get started with Python?",
        }
    ]
)

print(f"Thread ID: {thread.id}")

run = openai.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)

print(f"Run ID: {run.id}")


## RAG Assistant
* Changes RAG: not necessary for simple cases.
* RAG still necessary for complex and sophisticated cases.

In [ ]:
import openai

file = openai.files.create(
    file=openai.file_from_path("file.pdf"),
    purpose="assistants",
)

assistant = openai.beta.assistants.create(
    model="gpt-3.5-turbo-1106",
    file_ids=[
        file.id,
    ],
    tools=[
        {"type": "retrieval"},
    ]
)

thread = openai.beta.threads.create()

print(f"File ID: {file.id}")
print(f"Assistant ID: {assistant.id}")
print(f"Thread ID: {thread.id}\n")

print("GPT: Hello! How can I assist you today?")

while True:
    message = input("You: ")
    print()

    openai.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=message,
    )

    run = openai.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant.id,
    )

    while True:
        run = openai.beta.threads.runs.retrieve(
            run_id=run.id,
            thread_id=thread.id,
        )

        if run.status not in ["queued", "in_progress", "cancelling"]:
            break

    messages = openai.beta.threads.messages.list(
        thread_id=thread.id,
        limit=1,
    )
    print("GPT: " + messages.data[0].content[0].text.value)


## Multi-modality: Vision

In [ ]:
import openai
import base64

def encode_image(image_path):
    with open(image_path, "rb") as f:
        return base64.b64encode(f.read()).decode()

base64_image = encode_image("test.jpg")

response = openai.chat.completions.create(
    model="gpt-4-vision-preview",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "What's in this image?",
                },
                {
                    "type": "image_url",
                    "image_url": f"data:image/jpeg;base64,{base64_image}",
                }
            ]
        }
    ]
)

print(response.choices[0].message.content)


## Multi-modality: Audio (TextSpeech)

In [ ]:
import playsound
import openai
import json
import os

def play_with_voice(voice, speech):
    audio = openai.audio.speech.create(
        input=speech,
        model="tts-1",
        voice=voice,
    )

    audio.stream_to_file("audio.mp3")
    playsound.playsound("audio.mp3")
    os.remove("audio.mp3")

def mark_speaks(speech):
    play_with_voice("echo", speech)

def lisa_speaks(speech):
    play_with_voice("nova", speech)

response = openai.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=[
        {
            "role": "system",
            "content": "You are a conversation generator. Generate the full conversation using function calls. Call the speech functions 5 times for both Mark and Lisa",
        },
        {
            "role": "user",
            "content": "Create a conversation between Lisa and Mark. They're talking about Rust vs C++"
        }
    ],
    tools=[
        {
            "type": "function",
            "function": {
                "name": "mark_speaks",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "speech": {
                            "type": "string"
                        }
                    }
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "lisa_speaks",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "speech": {
                            "type": "string"
                        }
                    }
                }
            }
        }
    ]
)

for tool_call in response.choices[0].message.tool_calls:
    function_name = tool_call.function.name
    arguments = json.loads(tool_call.function.arguments)

    print(function_name + ": " + arguments["speech"])

    if function_name in ["mark_speaks", "lisa_speaks"]:
        globals()[function_name](**arguments)
